##Ductile Micro Grinding - Extended
#Inputs

Dc is tool diameter

Ae is radial stepover

Cc is the grain concentration according to FEPA. C145 for ordinary metal bond, C75 for ordinary resin bond from Haefeli

dg is the average grain size in microns, i.e. D33 = 33 microns

E is the Modulus of Elasticity, H is git the hardness and K_c is the toughness

Vc is tool surface speed

Critical chip equations are according to Bifano (1987-1991) and Huang (2020)

Bifano $$d_c=0.15\left(\frac{E}{H}\right)\left(\frac{K_{C}}{H}\right)^2$$ 

Huang $$d_c=8.7\left(\frac{H}{E}\right)^{1/2}\left(\frac{K_{C}}{H}\right)^2$$

Maximum critical chip thickness for both is $$h_{cu-max}=\sqrt{\frac{4}{C\cdot r}\frac{v_w}{v_c}\sqrt{\frac{a_e}{d_{eq}}}}$$

In [28]:
import math
import pint
import ipywidgets as widgets
ureg = pint.UnitRegistry()


slider_rpm = widgets.IntSlider(value=100000, min=10, max=250000, description='Spindle rpm')
slider_diameter = widgets.FloatSlider(value=3, min=0.1, max=10, description='Dc mm')

slider_chip_aspect_ratio = widgets.IntSlider(value=10, min=2, max=50, description='r chip ratio')
slider_grain_density = widgets.FloatLogSlider(value=500, base=10, min=1, max=6, step=0.001, description='C grain density')

slider_E = widgets.IntSlider(value=168, min=10, max=1000, step=1, description='E GPa')
slider_H = widgets.FloatSlider(value=11, min=1, max=80, step=0.1, description='H GPa')
slider_Kc = widgets.FloatSlider(value=0.7, min=0.1, max=20, step=0.1, description='Kc MPa.m^1/2')

slider_stepover = widgets.FloatLogSlider(value=0.001, base=10, min=-6, max=1, step = 0.01,description='Ae mm')

output_crit_chip_bifano = widgets.FloatText(disabled=True, description='Bi h_cu-max')
output_crit_chip_huang = widgets.FloatText(disabled=True, description='Hu h_cu-max')
output_surface_speed = widgets.FloatText(disabled=True, description='Vc m/min')
output_feedrate_side_bifano = widgets.FloatText(disabled=True, description='Bi Vf mm/s')
output_feedrate_side_huang = widgets.FloatText(disabled=True, description='Hu Vf mm/s')


def compute_surface_speed(*ignore):
    output_surface_speed.value = round(math.pi*slider_diameter.value * slider_rpm.value/1000)

def compute_crit_chip(*ignore):
    E = slider_E.value*ureg.gigapascal
    H = slider_H.value*ureg.gigapascal
    Kc = slider_Kc.value*ureg.megapascal
    lambda_bifano = 0.15
    lambda_huang = 8.7

    output_crit_chip_bifano.value = lambda_bifano*(E/H)*((Kc/H)**2)
    output_crit_chip_huang.value = lambda_huang*(math.sqrt(H/E))*((Kc/H)**2)

def compute_feedrates_crit_chip(*ignore):
    surface_speed = output_surface_speed.value#*ureg.meter/ureg.second
    crit_chip_bifano = output_crit_chip_bifano.value#*ureg.meter
    crit_chip_huang = output_crit_chip_huang.value#*ureg.meter
    grain_density = slider_grain_density.value
    chip_aspect_ratio = slider_chip_aspect_ratio.value
    stepover = slider_stepover.value
    diameter = slider_diameter.value

    output_feedrate_side_bifano.value = round((surface_speed*(1000*crit_chip_bifano)**2)/((4/(grain_density*chip_aspect_ratio))*math.sqrt(stepover/diameter)),4)

    output_feedrate_side_huang.value = round((surface_speed*(1000*crit_chip_huang)**2)/((4/(grain_density*chip_aspect_ratio))*math.sqrt(stepover/diameter)),4)


slider_rpm.observe(compute_surface_speed, 'value')
slider_diameter.observe(compute_surface_speed, 'value')

slider_E.observe(compute_crit_chip, 'value')
slider_H.observe(compute_crit_chip, 'value')
slider_Kc.observe(compute_crit_chip, 'value')

output_surface_speed.observe(compute_feedrates_crit_chip, 'value')
slider_diameter.observe(compute_feedrates_crit_chip, 'value')
slider_chip_aspect_ratio.observe(compute_feedrates_crit_chip, 'value')
slider_grain_density.observe(compute_feedrates_crit_chip, 'value')
slider_stepover.observe(compute_feedrates_crit_chip, 'value')
output_crit_chip_bifano.observe(compute_feedrates_crit_chip, 'value')
output_crit_chip_huang.observe(compute_feedrates_crit_chip, 'value')


display(slider_rpm)
display(slider_diameter)
display(slider_chip_aspect_ratio)
display(slider_grain_density)
display(slider_E)
display(slider_H)
display(slider_Kc)
display(slider_stepover)




IntSlider(value=100000, description='Spindle rpm', max=250000, min=10)

FloatSlider(value=3.0, description='Dc mm', max=10.0, min=0.1)

IntSlider(value=10, description='r chip ratio', max=50, min=2)

FloatLogSlider(value=500.0, description='C grain density', max=6.0, min=1.0, step=0.001)

IntSlider(value=168, description='E GPa', max=1000, min=10)

FloatSlider(value=11.0, description='H GPa', max=80.0, min=1.0)

FloatSlider(value=0.7, description='Kc MPa.m^1/2', max=20.0, min=0.1)

FloatLogSlider(value=0.001, description='Ae mm', max=1.0, min=-6.0, step=0.01)

#Results

In [29]:
display(output_surface_speed)

display(output_crit_chip_bifano)
display(output_crit_chip_huang)

display(output_feedrate_side_bifano)
display(output_feedrate_side_huang)


FloatText(value=0.0, description='Vc m/min', disabled=True)

FloatText(value=9.277235161532683e-09, description='Bi h_cu-max', disabled=True)

FloatText(value=9.015123156656094e-09, description='Hu h_cu-max', disabled=True)

FloatText(value=0.0, description='Bi Vf mm/s', disabled=True)

FloatText(value=0.0, description='Hu Vf mm/s', disabled=True)